# Coleta de dados dos municípios brasileiros

- Fontes: https://cidades.ibge.gov.br/ - IBGE


- Objetivo: Coletar os dados disponíveis no portal do IBGE referente ao último censo (na data de criação deste notebook, trata-se do censo de 2010), dos municípios brasileiros, e os salvar em um .csv, devidamente limpo e formatado.

## Por que Selenium?

O conteúdo do Portal do IBGE é gerado de forma dinâmica, sendo que algumas informações estão "escondidas" e requerem interações do usuário (movimentos do mouse, cliques, etc) para serem disponibilizadas. O Selenium permite simular o comportamento de usuários humanos, expondo as informações relevantes e permitindo, por fim, que se faça a coleta dos dados.

## Importando as bibliotecas e definindo constantes e variáveis iniciais

> Para que o script funcione é necessário estar com o Selenium devidamente instalado, com o *chromedriver*. Para mais informações sobre a instalação do Selenium, confira a documentação [aqui](https://selenium-python.readthedocs.io/installation.html).

In [18]:
from selenium.webdriver import Chrome
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support.expected_conditions import (
    presence_of_element_located,
    element_to_be_clickable
)
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep

PORTAL = "https://cidades.ibge.gov.br"

webdriver = Chrome()
webdriverwait = WebDriverWait(webdriver, 60)

data = []

## Função para coleta das informações

Quando a página de um muncípio for acessada, é necessário que as informações sejam coletadas. Isto é definido na função a seguir, de modo que:
- "cidade" = nome da cidade.
- "estado" = nome do estado.
- "populacao" = população da cidade no último censo.
- "densidade" = densidade demográfica da cidade, em habitante por km².
- "salario_medio" = salário médio mensal dos trabalhadores da cidade, em salários mínimos.
- "populacao_ocupada" = número de pessoas classificadas como ocupadas.
- "ate_meio_salario" = percentual da população da cidade com renda nominal mensal de até meio salário mínimo.
- "escolarizacao" = percentual da escolarização de crianças entre 6 e 14 anos na cidade.
- "pib" = PIB per capita da cidade.
- "receitas_fontes_externas" = percentual de receitas oriundas de fontes externas.
- "receitas_total" = total de receitas realizadas (dividido por 1000).
- "despesas" = total de despesas empenhadas (dividido por 1000).
- "mortalidade_infantil" = número de óbitos por mil nascidos vivos.
- "internacoes_diarreia" = internações por diarréia, por mil habitantes.
- "area" = area da unidade territorial em km².
- "esgotamento_sanitario" = percentual de esgotamento sanitário adequado.
- "arborizacao" = percentual de arborização de vias públicas.
- "urbanizacao" = percentual de urbanização de vias públicas.

> Já faço, neste momento, uma reformatada inicial nos dados coletados, editando as strings, removendo pontos, substituindo as vírgulas das casas decimais por pontos e, de modo geral, deixando as *strings* mais próximas dos devidos valores numéricos correspondentes. A conversão de *string* para números poderia ser feita aqui, mas preferi deixar para a etapa de limpeza e tratamento dos dados, a fim de diminuir a complexidade da seguinte função.

In [15]:
def collect_data(webdriver):
    page_values = webdriver.find_elements_by_css_selector('.indicador__valor')
    city_data = {
        "cidade": webdriver.find_element_by_css_selector('div#local h1').text,
        "estado": webdriver.find_elements_by_css_selector('div#local a')[1].text,
        "populacao": page_values[0].text.split()[0].replace('.', ''),
        "densidade": page_values[1].text.split()[0].replace('.', '').replace(',', '.'),
        "salario_medio": page_values[2].text.split()[0].replace(',', '.'),
        "populacao_ocupada": page_values[3].text.split()[0].replace('.', ''),
        "ate_meio_salario": page_values[5].text.split()[0].replace(',', '.'),
        "escolarizacao": page_values[6].text.split()[0].replace(',', '.'),
        "pib": page_values[9].text.split()[0].replace('.', '').replace(',', '.'),
        "receitas_fontes_externas": page_values[10].text.split()[0].replace(',', '.'),
        "receitas_total": page_values[11].text.split()[0].replace('.', '').replace(',', '.'),
        "despesas": page_values[12].text.split()[0].replace('.', '').replace(',', '.'),
        "mortalidade_infantil": page_values[13].text.split()[0].replace(',', '.'),
        "internacoes_diarreia": page_values[14].text.split()[0].replace(',', '.'),
        "area": page_values[15].text.split()[0].replace('.', '').replace(',', '.'),
        "esgotamento_sanitario": page_values[16].text.split()[0].replace(',', '.'),
        "arborizacao": page_values[17].text.split()[0].replace(',', '.'),
        "urbanizacao": page_values[18].text.split()[0].replace(',', '.')
    }
    return city_data

## Coleta dos dados

- Comportamento esperado do script:
    - Entrar na página inicial do portal.
    - Acessar a lista de estados brasileiros.
    - Para cada estado:
        - Acessar a lista de municípios do estado.
        - Para cada município:
            - Coletar o link da página do município.
    - Para cada link:
        - Acessar a página.
        - Coletar os dados.

In [ ]:
# Get to the main page and expose the state menu:

webdriver.get(PORTAL)

webdriverwait.until(
    presence_of_element_located((By.CSS_SELECTOR, '.aside_recolhido'))
)
action_chain = ActionChains(webdriver)
action_chain.move_to_element(
    webdriver.find_element_by_css_selector('.aside_recolhido')
).perform()

webdriverwait.until(
    element_to_be_clickable((By.CSS_SELECTOR, 'div#localidade button'))
).click()

webdriverwait.until(
    element_to_be_clickable((By.CSS_SELECTOR, 'li#menu__municipio'))
).click()

# Loop to go through each state:

webdriverwait.until(
    presence_of_element_located((By.CSS_SELECTOR, 'div#segunda-coluna'))
)
states = webdriver.find_elements_by_css_selector('div#segunda-coluna li')
states = states[1:] # Get rid of the 'Todos' option

links_list = []
for state in states:
    webdriverwait.until(
        element_to_be_clickable((By.CSS_SELECTOR, 'div#segunda-coluna li'))
    )
    state.click()
    
    # Loop through each city and collect link to city page
    webdriverwait.until(
        presence_of_element_located((By.CSS_SELECTOR, 'div.por-estado__selecionar-municipio'))
    )
    citys = webdriver.find_elements_by_css_selector('div.municipios a')
    
    for city in citys:
        link = city.get_attribute('href')
        links_list.append(link)
        
# Loops through each city link and collects data

timeout = 0
for link in links_list:
    # A high number of accesses, in a small amount of time, may cause service instability.
    # The following sleep statement diminishes the issue, waiting 5 min after every 100 accesses.
    if not timeout % 100:
        sleep(300)
        
    webdriver.get(link)
    webdriverwait.until(
        presence_of_element_located((By.CSS_SELECTOR, '.indicador__valor'))
    )
    data.append(
        collect_data(webdriver)
    )